# **Label every cluster**

### **1. Import**

In [1]:
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from collections import defaultdict
import sys
import os
import json
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np, pandas as pd, re, random
from tqdm.auto import tqdm
# Level up one level directory to add app the the allowed routes
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from app.utils.base_dir import BASE_DIR

w:\PROFESIONAL\pmb-clusterign\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **2. Load the model**

In [2]:
#embed_model = SentenceTransformer("allenai/scibert_scivocab_uncased")
embed_model = SentenceTransformer("all-MiniLM-L6-v2") # using a mini model for performance
kw_model = KeyBERT(model=embed_model) 

w:\PROFESIONAL\pmb-clusterign\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rofer\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back t

### **3. Extract the keywords for cluster**

**3.1 Function to extract the keywords per cluster**

In [ ]:
def extract_keywords_keybert_clusterwise(
        texts,  # the corpus of the cluster
        top_n=10,       #number of keywords that return
        keyphrase_ngram_range=(1,2),    # use bigrams
        use_mmr=True,   # avoid redudant words
        diversity=0.6,  # similitud and viriety
        nr_candidates=20,
        stop_words='english',
    ):
    
    kws = kw_model.extract_keywords(texts,
        keyphrase_ngram_range = keyphrase_ngram_range,
        stop_words = stop_words,
        top_n = top_n,
        use_mmr = use_mmr,
        diversity = diversity,
        nr_candidates = nr_candidates)
    # kws -> list of (keyword, score)
    kws = [k for k,_ in kws]
    return kws
    

**3.2 Load the data**

In [4]:
no_clean_data_path = BASE_DIR/'data'/'processed'/'noCleanProcessedData.json' 
clusters_path = BASE_DIR/'data'/'processed'/'clusters.json' 

no_clean_data_array = []
no_clean_data = {}
clusters = {}
with open(no_clean_data_path, 'r') as f:
    no_clean_data_array = json.load(f)

with open(clusters_path, 'r') as f:
    clusters = json.load(f)

def parse_to_dict(data:list):
    for element in data:
        for k, v in element.items():
            no_clean_data[k] = v
parse_to_dict(no_clean_data_array)



**3.3 Get cluster corpus**

In [1]:
# like {11 : ["text 01", "text 02"]}
clusterNumber_corpusText = defaultdict(str)
MAX_DOCS = 20

for cluster_number, titles_list in clusters.items():
    for i, title in enumerate(titles_list):
        i+=1
        if i <= MAX_DOCS:
            clusterNumber_corpusText[cluster_number] += no_clean_data[title]
        else: 
            break

NameError: name 'defaultdict' is not defined

**3.2 Extract keywords for all custer's**

In [ ]:
def extract_keywords():
    response = {} # is a dict with {number_cluster:[keyword01, keyword02,...]}
    for c_number, list_text in clusterNumber_corpusText.items():
        kwrds = extract_keywords_keybert_clusterwise(list_text)
        response[c_number] = kwrds 
    return response

res = extract_keywords()